<a href="https://colab.research.google.com/github/FabricioLealRS/potencial_clinicas_fisioterapia/blob/Code/Trabalho_grupo_marketing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os

# Diretório dos arquivos
diretorio = 'C:/Users/mariovaldobf/Desktop/Importação/'

# Lista dos nomes dos arquivos
arquivos = [
    'Acidentes_de_transito.csv',
    'Estabelecimentos.csv',
    'Municipios.csv',
    'Populacao_residente.csv',
    'Procedimentos_hospitalares_ARTICULACOES.csv',
    'Procedimentos_hospitalares_AVC.csv',
    'Procedimentos_hospitalares_CONDICOESNAUROLOGICAS.csv',
    'Procedimentos_hospitalares_DOENCASCRONICAS.csv',
    'Procedimentos_hospitalares_ORTOPEDICAS.csv',
    'Qtd_beneficiarios_planosdesaude.csv'
]

# Dicionário que vai armazenar os dataframes
dfs = {}

for arquivo in arquivos:
    # Cria o caminho completo do arquivo
    caminho_completo = os.path.join(diretorio, arquivo)

    # Lê o arquivo CSV
    df = pd.read_csv(caminho_completo, sep=';')

    # Remove a extensão .csv do nome do arquivo para usar como chave
    chave = os.path.splitext(arquivo)[0]

    # Armazena o dataframe no dicionário
    dfs[chave] = df

# Agora você pode acessar cada dataframe através da chave correspondente
# Por exemplo, para ver as primeiras linhas do dataframe 'Acidentes_de_transito', você pode fazer:
dfs['Acidentes_de_transito'].head(10)


**União dos Dataframes de Procedimentos Hospitalares**

Primeiramente iremos realizar a união dos dataframes de procedimento hospitalar, esse passo é necessário, pois embora não seja possivel baixarmos um unica arquivo no site do governo, para melhor trabalho podemos realizar a união destes todos em um unico df chamado df_procedimentos_hospitalaresPRE

In [ ]:
# Lista dos dataframes a serem unidos
dfnames = [
    'Procedimentos_hospitalares_ARTICULACOES',
    'Procedimentos_hospitalares_AVC',
    'Procedimentos_hospitalares_CONDICOESNAUROLOGICAS',
    'Procedimentos_hospitalares_DOENCASCRONICAS',
    'Procedimentos_hospitalares_ORTOPEDICAS'
]

# Inicializa o dataframe final com o primeiro dataframe da lista
df_procedimentos_hospitalares = dfs[dfnames[0]].rename(columns={'AIH_aprovadas': dfnames[0]})

# Itera sobre o resto dos dataframes
for name in dfnames[1:]:
    # Renomeia a coluna 'AIH_aprovadas' para o nome do dataframe e realiza a união
    df_procedimentos_hospitalares = df_procedimentos_hospitalares.merge(dfs[name].rename(columns={'AIH_aprovadas': name}), on='Municipio', how='outer')

# Remove a string 'Procedimentos_hospitalares_' dos nomes das outras colunas
df_procedimentos_hospitalares.columns = df_procedimentos_hospitalares.columns.str.replace('Procedimentos_hospitalares_', '')

# Split the 'Municipio' column into 'Codigo' and 'Nome'
df_procedimentos_hospitalaresPRE['Codigo'] = df_procedimentos_hospitalaresPRE['Municipio'].str.slice(0, 6)
df_procedimentos_hospitalaresPRE['Nome'] = df_procedimentos_hospitalaresPRE['Municipio'].str.slice(7)

# Remove the original 'Municipio' column
df_procedimentos_hospitalaresPRE = df_procedimentos_hospitalaresPRE.drop(columns=['Municipio'])

# Reorder the columns to put 'Codigo' and 'Nome' first
df_procedimentos_hospitalaresPRE = df_procedimentos_hospitalaresPRE[['Nome', 'Codigo'] + [col for col in df_procedimentos_hospitalaresPRE.columns if col not in ['Codigo', 'Nome']]]

# Salva o dataframe ajustado em um arquivo CSV no diretório especificado
df_procedimentos_hospitalares.to_csv(diretorio + 'df_procedimentos_hospitalares.csv', index=False, sep=';')

# Exibe as primeiras linhas do dataframe final
df_procedimentos_hospitalares.head()

**Ajuste do df de população residente**

In [ ]:
# Access the 'Populacao_residente' dataframe
df_populacao = dfs['Populacao_residente']

# Print the first few rows to understand its structure
df_populacao.head()

# Drop the 'total' column
df_populacao = df_populacao.drop(columns=['Total'])

# Split the 'Municipio' column into 'Codigo' and 'Nome'
df_populacao['Codigo'] = df_populacao['Municipio'].str.slice(0, 6)
df_populacao['Nome'] = df_populacao['Municipio'].str.slice(7)

# Drop the 'Municipio' column
df_populacao = df_populacao.drop(columns=['Municipio'])

# Reorder the columns to put 'Codigo' and 'Nome' at the beginning
df_populacao = df_populacao[['Nome', 'Codigo'] + [col for col in df_populacao.columns if col not in ['Codigo', 'Nome']]]

# Cria as novas colunas somando as colunas correspondentes
df_populacao['Menor que 1 a 9 anos'] = df_populacao['Menor 1 ano'] + df_populacao['1 a 4 anos'] + df_populacao['5 a 9 anos']
df_populacao['10 a 19 anos'] = df_populacao['10 a 14 anos'] + df_populacao['15 a 19 anos']

# Agora, se você desejar, pode excluir as colunas originais que foram somadas
df_populacao = df_populacao.drop(columns=['Menor 1 ano', '1 a 4 anos', '5 a 9 anos', '10 a 14 anos', '15 a 19 anos'])


# Save the adjusted dataframe to a CSV file in the specified directory
df_populacao.to_csv(diretorio + 'df_populacao.csv', index=False, sep=';')

# Print the updated dataframe
df_populacao.head()

**Ajuste do df de acidente de transito**

In [ ]:
# Access the 'Acidentes_de_transito' dataframe
df_acidentes = dfs['Acidentes_de_transito']

# Split the 'Município de ocorrência' column into 'Codigo' and 'Nome'
df_acidentes['Codigo'] = df_acidentes['Município de ocorrência'].str.slice(0, 6)
df_acidentes['Nome'] = df_acidentes['Município de ocorrência'].str.slice(7)

# Drop the 'Município de ocorrência' column
df_acidentes = df_acidentes.drop(columns=['Município de ocorrência'])

# Rename the 'Total' column to 'Acidentes de transito'
df_acidentes = df_acidentes.rename(columns={'Total': 'Acidentes de transito'})

# Reorder the columns to put 'Codigo' and 'Nome' first
df_acidentes = df_acidentes[['Nome', 'Codigo'] + [col for col in df_acidentes.columns if col not in ['Codigo', 'Nome']]]

# Print the updated dataframe
df_acidentes.head()

# Save the adjusted dataframe to a CSV file in the specified directory
df_acidentes.to_csv(diretorio + 'df_acidentes.csv', index=False, sep=';')


**Ajuste do df de Quantidade de Beneficiários do Plano de Saúde**

In [ ]:
# Access the 'Qtd_beneficiarios_planosdesaude' dataframe
df_planos = dfs['Qtd_beneficiarios_planosdesaude']

# Split the 'Município' column into 'Codigo' and 'Nome'
df_planos['Codigo'] = df_planos['Município'].str.slice(0, 6)
df_planos['Nome'] = df_planos['Município'].str.slice(7)

# Drop the 'Município', 'Inconsistente' and 'Total' columns
df_planos = df_planos.drop(columns=['Município', 'Inconsistente', 'Total'])

# Create new columns by summing the desired columns
df_planos['Menor que 1 a 9 anos'] = df_planos['Até 1 ano'] + df_planos['1 a 4 anos'] + df_planos['5 a 9 anos']
df_planos['10 a 19 anos'] = df_planos['10 a 14 anos'] + df_planos['15 a 19 anos']
df_planos['20 a 29 anos'] = df_planos['20 a 24 anos'] + df_planos['25 a 29 anos']
df_planos['30 a 39 anos'] = df_planos['30 a 34 anos'] + df_planos['35 a 39 anos']
df_planos['40 a 49 anos'] = df_planos['40 a 44 anos'] + df_planos['45 a 49 anos']
df_planos['50 a 59 anos'] = df_planos['50 a 54 anos'] + df_planos['55 a 59 anos']
df_planos['60 a 69 anos'] = df_planos['60 a 64 anos'] + df_planos['65 a 69 anos']
df_planos['70 a 79 anos'] = df_planos['70 a 74 anos'] + df_planos['75 a 79 anos']

# Rename the '80 anos ou mais' column
df_planos = df_planos.rename(columns={'80 anos ou mais': '80 anos e mais'})

# Drop the original columns
df_planos = df_planos.drop(columns=['Até 1 ano', '1 a 4 anos', '5 a 9 anos', '10 a 14 anos', '15 a 19 anos',
                                    '20 a 24 anos', '25 a 29 anos', '30 a 34 anos', '35 a 39 anos', '40 a 44 anos',
                                    '45 a 49 anos', '50 a 54 anos', '55 a 59 anos', '60 a 64 anos', '65 a 69 anos',
                                    '70 a 74 anos', '75 a 79 anos'])

# Reorder the columns to put 'Codigo' and 'Nome' first
df_planos = df_planos[['Nome', 'Codigo'] + [col for col in df_planos.columns if col not in ['Codigo', 'Nome']]]

# Print the updated dataframe
df_planos.head()

# Save the adjusted dataframe to a CSV file in the specified directory
df_planos.to_csv(diretorio + 'df_planos.csv', index=False, sep=';')


**Ajuste do df de Estabelecimentos**

In [ ]:
df_empresas = dfs['Estabelecimentos']

# List of columns to drop
cols_to_drop = [
    'CNPJ BÃƒÂSICO',
    'CNPJ ORDEM',
    'CNPJ DV',
    'IDENTIFICADOR M-F',
    'NOME FANTASIA',
    'MOTIVOSITUACAO',
    'NOME CIDADE EXTERIOR',
    'PAIS',
    'CNAE SECUNDARIO',
    'TIPO LOGRADOURO',
    'LOGRADOURO',
    'NUMERO',
    'COMPLEMENTO',
    'BAIRRO',
    'CEP',
    'DDD1',
    'TELEFONE1',
    'DDD2',
    'TELEFONE2',
    'DDD FAX',
    'FAX',
    'CORREIO',
    'SITUACAO ESPECIAL',
    'DATASITUCACAO ESPECIAL',
    'CNPJ BÃSICO'
]

# Drop the columns
df_empresas = df_empresas.drop(columns=cols_to_drop)

# Show the first few rows of the updated DataFrame
df_empresas.head()


In [ ]:
# Create a series with the 'MUNICIPIO' column as index and the 'NOME MUNICIPIO' column as values
nome_municipios = dfs['Municipios'].set_index('MUNICIPIO')['NOME MUNICIPIO']

# Replace the values in the 'MUNICIPIO' column of the 'Estabelecimentos' dataframe using the created series
df_empresas['MUNICIPIO'] = df_empresas['MUNICIPIO'].map(nome_municipios)

# Show the first few rows of the updated DataFrame
df_empresas.head()


In [ ]:
# Dictionary to replace values in the 'SITUACAO CADASTRAL' column
situacao_cadastral_map = {
    1: 'NULA',
    2: 'ATIVA',
    3: 'SUSPENSA',
    4: 'INAPTA',
    8: 'BAIXADA'
}

# Replace the values in the 'SITUACAO CADASTRAL' column using the dictionary
df_empresas['SITUACAO CADASTRAL'] = df_empresas['SITUACAO CADASTRAL'].map(situacao_cadastral_map)

# Show the first few rows of the updated DataFrame
df_empresas.head()


In [ ]:
from datetime import datetime

# Exclude rows with 'NULA', 'SUSPENSA', 'INAPTA', and 'BAIXADA' values in the 'SITUACAO CADASTRAL' column
df_empresas = df_empresas[~df_empresas['SITUACAO CADASTRAL'].isin(['NULA', 'SUSPENSA', 'INAPTA', 'BAIXADA'])]

# Exclude the 'DATA SITUACAO' column
df_empresas = df_empresas.drop(columns=['DATA SITUACAO'])

# Exclude the 'SITUACAO CADASTRAL' column
df_empresas = df_empresas.drop(columns=['SITUACAO CADASTRAL'])

# Exclude the 'CNAE PRINCIPAL' column
df_empresas = df_empresas.drop(columns=['CNAE PRINCIPAL'])


In [ ]:
from math import floor

# Convert 'DATA INICIO ATIVIDADE' column to string type
df_empresas['DATA INICIO ATIVIDADE'] = df_empresas['DATA INICIO ATIVIDADE'].astype(str)

# Add slashes in 'DATA INICIO ATIVIDADE' column
df_empresas['DATA INICIO ATIVIDADE'] = df_empresas['DATA INICIO ATIVIDADE'].apply(lambda x: f"{x[:4]}/{x[4:6]}/{x[6:]}")

# Convert 'DATA INICIO ATIVIDADE' column to datetime type
df_empresas['DATA INICIO ATIVIDADE'] = pd.to_datetime(df_empresas['DATA INICIO ATIVIDADE'])

# Calculate activity time in years without decimals
df_empresas['TEMPO DE ATIVIDADE (ANOS)'] = (datetime.now() - df_empresas['DATA INICIO ATIVIDADE']).dt.days / 365
df_empresas['TEMPO DE ATIVIDADE (ANOS)'] = df_empresas['TEMPO DE ATIVIDADE (ANOS)'].apply(floor)

# Print the first few rows of the updated DataFrame
print(df_empresas.head())

# Save the adjusted DataFrame to a CSV file in the specified directory
df_empresas.to_csv(diretorio + 'df_empresas.csv', index=False, sep=';')


In [ ]:
# Create the DataFrame df_empresas_qtd with the count of municipalities
df_empresas_qtd = pd.DataFrame(df_empresas['MUNICIPIO'].value_counts()).reset_index()

# Rename the columns
df_empresas_qtd.columns = ['MUNICIPIO', 'Qtd empresas']

# Print the DataFrame df_empresas_qtd
print(df_empresas_qtd)

# Save the adjusted DataFrame to a CSV file in the specified directory
df_empresas_qtd.to_csv(diretorio + 'df_empresas_qtd.csv', index=False, sep=';')
